## Mechanical Tests

Mechanical tests check the an implementation without regards for the science that underpins the code. This approach is often used when testing smaller chunks of code in isolation. The test can be given physically non-sensical input, and return unphysical outputs, as long as the flow of data and data transformations are the same. Lets go directly to a simple example.

### Easy but unphysical input

When examining fluid flows, we are interested the momentum $\mu$ at each lattice site. It can be obtained as the sum over the momentum of the particles at the site. However, lets promptly forget about the physics. Here is what the function looks like:

In [2]:
momentum(fᵢ::DenseVector, cᵢ::DenseMatrix) = cᵢ * fᵢ

momentum (generic function with 1 method)

We could try and setup physical intputs for this function and test the result. But it is much simpler to check using short integer inputs, rathe than the longuer floating points vectors and matrices the function will be given during an actual Lattice-Boltzmann calculation.

In [3]:
using FactCheck: @fact, facts
facts("Check momentum function over single site") do
    @fact momentum([1, 1], [1 2; 3 4]) => [3, 7]
    @fact momentum([-1, 1], [1 2; 3 4]) => [1, 1]
end
nothing

Check momentum function over single site
2 facts verified.


Sure, the `momentum` function is short and simple. But cutting the code down to size is what unit-testing is all about. And using input crafted for simplicity rather than physicality will get a lot of mileage in more complex situations as well.

### Comparison with (simplified) test algorithm

It is often possible to implement an algorithm several ways. For instance, when implementing the derivative of some function, it is a good idea to test it against the numerical derivatives, i.e. the comparison $\frac{\partial f(x)}{\partial x} \approx \frac{f(x+\delta x) - f(x)}{\delta x}$. Implementing a functionality twice might seem like a waste of time, but in practice it proves often invaluable. It is much easier to understand where a bug has crept in when there is alternate implementation that can be queried for intermediate values (e.g. value of some variable `x` halfway through the code). It is true that during the implementation phase, when neither the target code nor the test are known to work, it is not always clear which is responsible for a discrepancy. But it is a price worth paying in practice.

This is how the function giving the fluid density $\rho$ at a site $\vec{r}$ is tested in LatBo. The density is defined simply as the sum over the number of particles at the site. The magic is that if the input that it is fed is a series, then we can use the result for a geometric series as a secondary algorithm $\sum_{i=1}^Ni = 0.5N(N+1)$: 

In [11]:
using FactCheck: @fact, facts

density(fᵢ::DenseVector) = sum(fᵢ)

facts("Check density by comparing with geometric series") do
    series(N) = N * (N+1)/2
    @fact density([1:10]) => series(10)
    @fact density([10:15]) => series(15) - series(9)
end
nothing

Check density by comparing with geometric series
2 facts verified.


We've done two things here: (i) the input is simplified to integers, (ii) the algorithm of `density` is compared to another, `series`. As always, one should be wary that the test covers all use cases, especially when comparing to a restricted algorithm as done here. Due to the simplicity of the original algorithm, there is little doube in this case that the test is sufficient.

### Checking for specific behaviours

When testing, it is all too easy to convince one self that the numbers spewed by a piece of code are correct, when infact it they are garbage. Testing for behaviours instead is more robust. What kind of behaviour will depend on the actual method, but, in the case of mathematical functions, linearity is fairly common, at least for some arguments.

The summarization of Lattice-Boltzmann at the beginning of this chapter described the collision step where particles are exchanged between different algorithms. At least within the single-relaxation time algorithm, this step is very simple:

In [29]:
collision(τ⁻¹::Number, fᵢ::DenseVector, feq::DenseVector) = τ⁻¹ * (feq - fᵢ)

collision (generic function with 2 methods)

We can (and should) test for each argument separately. The function is linear with respect to $\tau^{-1}$ when the other variables are fixed. It is also linear with respect to `feq` ($f_i$) when $f_i == 0$ (`feq == 0`). And it is anti-symetric with respect to `feq` and $f_i$. Hence, the following tests:

In [30]:
using FactCheck: facts, @fact, context
facts("Behaviour of the single-relaxation time collision operator") do
    context("Linear vs τ⁻¹") do
        @fact 2collision(1, [1], [0]) => collision(2, [1], [0])
        @fact 3collision(1, [0], [1]) => collision(3, [0], [1])
    end
    context("Linear vs fᵢ (and feq)") do
        f = rand(Int8, 10)
        @fact 2collision(1, f, zeros(f)) => collision(1, 2f, zeros(f))
        @fact 3collision(1, zeros(f), f) => collision(1, zeros(f), 3f)
    end
    context("Anti-symmetric vs fᵢ and feq") do
        fᵢ, feq = rand(Int8, 10), rand(Int8, 10)
        @fact collision(1, fᵢ, feq) => -collision(1, feq, fᵢ)
    end
    context("Lock-down the sign") do
        @fact collision(1, [1:10], zeros(Int8, 10)) .< 0 => all
    end
end
nothing

Behaviour of the single-relaxation time collision operator
     - Linear vs τ⁻¹
     - Linear vs fᵢ (and feq)
     - Anti-symmetric vs fᵢ and feq
     - Lock-down the sign
6 facts verified.


Note the last fact-check in the code above. It isn't there as much to test as it is to make the code a bit more future proof. It merely records the chosen sign convention. It is an easy bug to introduce when faffing with the code. And it is an easy bug to ward against, so why not just do it? It's two minutes now vs two days sometime later.  

### Checking the flow of code and data -- a.k.a. Mocking

Once the internal its of codes are tested, it is time to test the scaffold that  puts those bits together.  Mocking, e.g. replacing complex  functions with fakes  can be very useful  at this juncture.  It allows to  better separation  of concerns when  testing:  bugs introduced in internal methods will not invalidate  tests  that replace them with Mocks. It  makes for faster testing when the mocked code replaces computationally intensive routines. And most importantly, it  means focusing the tests of integration functions specifically on  what integration functions: mediate the functionalites of (more) internal routines into a whole.  In the spirit of that latter remark, Mocking is really about  analyzing the flow data  and transformations through a piece of code. 

Lets look at  an example. The code below  is  (a simplified version of) is the main  Lattice-Boltzman  kernel  applied to each lattice site:

In [ ]:
function local_kernel(kernel::FluidKernel, sim::Simulation, site::Integer)
    const feq = equilibrium(sim.populations[:, site], sim.lattice)
    sim.populations[:, site] += collision(kernel.collision, sim.populations[:, site], feq)
    for direction in 1:length(sim.lattice.weights)
        const to = neighboring_site(sim.lattice, site, direction)
        const streamer = get(kernel.streamers, sim.playground[to], NULLSTREAMER)
        streaming(streamer, sim, site, to, direction)
    end
end